<a href="https://colab.research.google.com/github/devyadav11/DA6401_ASSIGNMENT_1/blob/main/DA6401__Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
## Question-1

!pip install wandb
import wandb
from keras.datasets import fashion_mnist

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math


wandb.init(project = "DA6401_ASSIGNMENT_1", entity="devyadav11")   #  initialize a Weights & Biases (WandB) run

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()   # loading data


type_class = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']


image_list = []
class_list = []

for i in range(10):
  class_y = np.argmax(y_train == i)
  image = x_train[class_y,:,:]
  plt.figure(figsize=[12,5])
  plt.subplot( 2, 5, i+1)
  plt.title(type_class[i])
  plt.imshow(image)
  image_list.append(image)
  class_list.append(type_class[i])



wandb.log({"Question 1": [wandb.Image(img, caption=caption) for img, caption in zip(image_list, type_class)]})








Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f317748e910>> (for pre_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

BrokenPipeError: [Errno 32] Broken pipe

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f317748e910>> (for post_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

In [2]:


#Activation functions
def relu(x):
  return np.maximum(0, x)

def sigmoid(x):
  clip_x = np.clip(x, -500, 500)  # Clipping x to avoid overflow
  return 1 / (1 + np.exp(-clip_x))

def tanh(x):
  clip_x = np.clip(x, -500, 500)  # Clipping x for uniformity
  return np.tanh(clip_x)

def identity(x):
  return x



In [3]:
## Question-2:


import wandb
from keras.datasets import fashion_mnist

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

np.random.seed(41)
# Load dataset
(X_train, Y_train), (X_test, Y_test) = fashion_mnist.load_data()
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=42)

# Normalize and reshape images
X_train = X_train.reshape(-1, 28*28) / 255.0
X_test = X_test.reshape(-1, 28*28) / 255.0
X_val = X_val.reshape(-1, 28*28) / 255.0


class neural_network():
  def __init__(self, layers, learning_rate, weightDecay, batch_size, epochs, optimiser, activationFunc, beta = 0.1, beta_RMS = 0.9, isWandb = True):

    self.x_train = X_train
    self.y_train = Y_train
    self.x_val = X_val
    self.y_val = Y_val
    self.x_test = X_test
    self.y_test = Y_test






    self.epochs = epochs
    self.layers = layers                     # number of neurons and hidden layers (instance variable)
    self.beta = 0.1
    self.beta_RMS = 0.9
    self.beta1 = 0.9
    self.beta2 = 0.999
    self.epsilon = 1e-9
    self.optimiser = optimiser
    self.activationFunc = activationFunc
    self.weightDecay = weightDecay

    self.batch_size = batch_size
    self.learning_rate = learning_rate


    # Initialize isWandb to False by default
    self.isWandb = isWandb


    self.weights = []
    self.bias = []


    for i in range(1, len(layers)):
      weight = np.random.randn(layers[i], layers[i-1])
      self.weights.append(weight)
      bias = np.random.randn(layers[i], 1)
      self.bias.append(bias)


  def softmax(self, x):
    exp_x = np.exp(x - np.max(x))
    return exp_x / np.sum(exp_x)

  def loss(self, y_pred):                                        # here y
    return -np.log(y_pred)


  def one_hot_encodeing(self, y_true):
    one_hot = np.eye(10)[y_true]
    return one_hot




  def forward_prop(self, x, W, b):
      x = x.reshape(-1,1)
      self.hidden_layer_h = []
      self.hidden_layer_a = []
      W = self.weights
      b = self.bias

      for i in range(len(self.layers) - 2):
        h = 0
        a = np.dot(W[i], x) + b[i]
        if self.activationFunc == "sigmoid":
          h = sigmoid(a)
        elif self.activationFunc == "tanh":
          h = tanh(a)
        elif self.activationFunc == "relu":
          h = relu(a)
        #h =  self.sigmoid(a)
        x = h

        self.hidden_layer_h.append(h)
        self.hidden_layer_a.append(a)

      #for i in range(len(self.hidden_layer_a)):
        #print(f"hidden layers h{i} =  {self.hidden_layer_a[i]}")
        #print(f"hidden layers a{i} =  {self.hidden_layer_h[i]}")

      output_layer = np.dot(W[-1], self.hidden_layer_h[-1]) + b[-1]
      output = self.softmax(output_layer)


      return output








  def backword_prop(self, x , y, W, b):
    x_input = x.reshape(-1,1)
    W = self.weights
    b = self.bias
    dW = []
    db = []

    y_output = self.forward_prop(x = x, W = W, b = b)
    #print(self.y_output)

    y_pred = np.dot((y_output).T, self.one_hot_encodeing(y))     # y_pred is predicated value of output y corrosponding to true class


    dh_out = -(1/(y_pred + self.epsilon))*self.one_hot_encodeing(y)
    da_out = -(self.one_hot_encodeing(y) - (y_output).T)
    dh_out = dh_out.reshape(-1, 1)                                           # gradiant of loss with respect to output activation layer (h)
    da_out = da_out.T
    self.da_stored = []                                                      # gradiant of loss with respect to output preactivation layer (a)


    for i in reversed(range(len(W))):
      if i == 0:
         pass
      else:

       dw = np.dot(da_out, self.hidden_layer_h[i-1].T)
       dW.append(dw)
       db_temp = np.sum(da_out, axis=1, keepdims=True)
       db.append(db_temp)

       dh = np.dot((W[i]).T, da_out)
       if self.activationFunc == "sigmoid":
        dg = self.hidden_layer_h[i-1] * (1 - self.hidden_layer_h[i-1])
       elif self.activationFunc == "tanh":
        dg = (1 + self.hidden_layer_h[i-1]) * (1 - self.hidden_layer_h[i-1])
       elif self.activationFunc == "relu":
         dg = np.where(self.hidden_layer_a[i-1] > 0, 1, 0)


       da = dh*dg
       self.da_stored.append(da)


       da_out = da

    dw = np.dot(da_out, x_input.T)
    dW.append(dw)
    db1 = np.sum(da_out, axis=1, keepdims=True)
    db.append(db1)

    dW.reverse()
    self.da_stored.reverse()
    db.reverse()




    return dW, db


  def accuracyLoss(self, x, y, W, b):
    correct = 0
    loss = 0
    #for i in range(len(1, 1+self.layers)):
      #loss += (self.weightDecay / 2) * np.linalg.norm(W[i]) ** 2
    for i in range(len(x)):
      x_input = x[i].reshape(-1,1)
      y_true = y[i]
      y_output = self.forward_prop(x = x_input, W = W, b = b)
      y_pred_class = np.argmax(y_output)
      if y_pred_class == y_true:
        correct += 1
      y_pred = np.dot((y_output).T, self.one_hot_encodeing(y_true))
      loss += self.loss(y_pred + 1e-9)                                 #1e-9 is addeded bcuz in case if ypred = 0 then function doesnt blow up!!


    return correct*100/len(x), loss/len(x)






  def SGD(self):

    W, b = self.weights, self.bias
    iteration = 0

    for epoch in range(self.epochs):
      print(f'epoch {epoch + 1}/{self.epochs}')



      indices = np.arange(len(self.x_train))
      np.random.shuffle(indices)

      x_train_shuffle = self.x_train[indices]
      y_train_shuffle = self.y_train[indices]

      for i in range(0, len(x_train_shuffle), self.batch_size):
        x_batch = x_train_shuffle[i:i + self.batch_size]
        y_batch = y_train_shuffle[i:i + self.batch_size]

        # Initilise dw and db
        dW = [np.zeros_like(w) for w in W]
        db = [np.zeros_like(b_i) for b_i in b]

        for j in range(len(x_batch)):

          output = self.forward_prop(x_batch[j], W, b)
          dW_inter, db_inter = self.backword_prop(y = y_batch[j], x = x_batch[j], W = W, b = b)

          for i in range(len(W)):
            dW[i] += dW_inter[i]
            db[i] += db_inter[i]

       # Final dW and db
        dW = [dw / len(x_batch) for dw in dW]
        db = [db_i / len(x_batch) for db_i in db]

        du_W = self.beta

       #calculating W, b for 1 batch
        for i in range(len(W)):
          W[i] -= self.learning_rate * (dW[i])
          b[i] -= self.learning_rate * (db[i])

      # acccuracy and loss calculated after each epoch
      acuu, loss = self.accuracyLoss(W = W, b = b, x = self.x_train, y = self.y_train)
      v_acc, v_loss = self.accuracyLoss(W=  W, b = b, x = self.x_val, y = self.y_val)

      if self.isWandb == True:
        wandb.log({'accuracy': acuu})
        wandb.log({'loss': loss})
        wandb.log({'v_accuracy': v_acc})
        wandb.log({'v_loss': v_loss})
        wandb.log({'epoch': iteration})
      print(f" accuracy = {acuu}, Loss = {loss}, Validation_accuracy =  {v_acc}, Validation_loss =  {v_loss}")
      iteration += 1


    # True W, b after training

    return W, b




  def Momentam_GD(self):
    W, b = self.weights, self.bias
    iteration = 0
    du_W = [np.zeros_like(w) for w in self.weights]
    du_b = [np.zeros_like(b_i) for b_i in self.bias]

    for epoch in range(self.epochs):
      print(f'epoch {epoch + 1}/{self.epochs}')



      indices = np.arange(len(self.x_train))
      np.random.shuffle(indices)

      x_train_shuffle = self.x_train[indices]
      y_train_shuffle = self.y_train[indices]

      for i in range(0, len(x_train_shuffle), self.batch_size):
        x_batch = x_train_shuffle[i:i + self.batch_size]
        y_batch = y_train_shuffle[i:i + self.batch_size]

        # Initilise dw and db
        dW = [np.zeros_like(w) for w in W]
        db = [np.zeros_like(b_i) for b_i in b]

        for j in range(len(x_batch)):

          output = self.forward_prop(x_batch[j], W, b)
          dW_inter, db_inter = self.backword_prop(y = y_batch[j], x = x_batch[j], W = W, b = b)

          for i in range(len(W)):
            dW[i] += dW_inter[i]
            db[i] += db_inter[i]

       # average dw and db in one batch
        dW = [dw / len(x_batch) for dw in dW]
        db = [db_i / len(x_batch) for db_i in db]

        for i in range(len(W)):
          du_W[i] = self.beta*du_W[i] + dW[i]
          du_b[i] = self.beta*du_b[i] + db[i]




       #calculating W, b for 1 batch
        for i in range(len(W)):
          W[i] -= self.learning_rate * (du_W[i])
          b[i] -= self.learning_rate * (du_b[i])



      # acccuracy and loss calculated after each epoch
      acuu, loss = self.accuracyLoss(W = W, b = b, x = self.x_train, y = self.y_train)
      v_acc, v_loss = self.accuracyLoss(W=  W, b = b, x = self.x_val, y = self.y_val)

      if self.isWandb == True:
        wandb.log({'accuracy': acuu})
        wandb.log({'loss': loss})
        wandb.log({'v_accuracy': v_acc})
        wandb.log({'v_loss': v_loss})
        wandb.log({'epoch': iteration})
      print(f" accuracy = {acuu}, Loss = {loss}, Validation_accuracy =  {v_acc}, Validation_loss =  {v_loss}")
      iteration += 1


    # True W, b after training

    return W, b









  def Nestrov_Based_GD(self):
    W, b = self.weights, self.bias
    W_lookAhead, b_lookAhead = self.weights, self.bias
    iteration = 0
    du_W = [np.zeros_like(w) for w in self.weights]
    du_b = [np.zeros_like(b_i) for b_i in self.bias]

    for epoch in range(self.epochs):
      print(f'epoch {epoch + 1}/{self.epochs}')



      indices = np.arange(len(self.x_train))
      np.random.shuffle(indices)

      x_train_shuffle = self.x_train[indices]
      y_train_shuffle = self.y_train[indices]

      for i in range(0, len(x_train_shuffle), self.batch_size):
        x_batch = x_train_shuffle[i:i + self.batch_size]
        y_batch = y_train_shuffle[i:i + self.batch_size]

        # Initilise dw and db
        dW_lookAhead = [np.zeros_like(w) for w in W]
        db_lookAhead = [np.zeros_like(b_i) for b_i in b]

        for j in range(len(x_batch)):

          output = self.forward_prop(x_batch[j], W, b)
          dW_inter, db_inter = self.backword_prop(y = y_batch[j], x = x_batch[j], W = W_lookAhead, b = b_lookAhead)

          for i in range(len(W)):
            dW_lookAhead[i] += dW_inter[i]
            db_lookAhead[i] += db_inter[i]

       # average dw and db in one batch
        dW_lookAhead = [dw / len(x_batch) for dw in dW_lookAhead]
        db_lookAhead = [db_i / len(x_batch) for db_i in db_lookAhead]

        for i in range(len(W)):
          du_W[i] = self.beta*du_W[i] + dW_lookAhead[i]
          du_b[i] = self.beta*du_b[i] + db_lookAhead[i]




       #calculating W, b for 1 batch
        for i in range(len(W)):
          W[i] -= self.learning_rate * (du_W[i])
          b[i] -= self.learning_rate * (du_b[i])

        for i in range(len(W)):
          W_lookAhead[i] = W[i] - self.beta*dW_lookAhead[i]
          b_lookAhead[i] = b[i] - self.beta*db_lookAhead[i]



      # acccuracy and loss calculated after each epoch
      acuu, loss = self.accuracyLoss(W = W, b = b, x = self.x_train, y = self.y_train)
      v_acc, v_loss = self.accuracyLoss(W=  W, b = b, x = self.x_val, y = self.y_val)

      if self.isWandb == True:
        wandb.log({'accuracy': acuu})
        wandb.log({'loss': loss})
        wandb.log({'v_accuracy': v_acc})
        wandb.log({'v_loss': v_loss})
        wandb.log({'epoch': iteration})
      print(f" accuracy = {acuu}, Loss = {loss}, Validation_accuracy =  {v_acc}, Validation_loss =  {v_loss}")
      iteration += 1


    # True W, b after training

    return W, b






  def RMS_prop(self):
    W, b = self.weights, self.bias
    iteration = 0
    dv_W = [np.zeros_like(w) for w in self.weights]
    dv_b = [np.zeros_like(b_i) for b_i in self.bias]

    for epoch in range(self.epochs):
      print(f'epoch {epoch + 1}/{self.epochs}')



      indices = np.arange(len(self.x_train))
      np.random.shuffle(indices)

      x_train_shuffle = self.x_train[indices]
      y_train_shuffle = self.y_train[indices]

      for i in range(0, len(x_train_shuffle), self.batch_size):
        x_batch = x_train_shuffle[i:i + self.batch_size]
        y_batch = y_train_shuffle[i:i + self.batch_size]

        # Initilise dw and db
        dW = [np.zeros_like(w) for w in W]
        db = [np.zeros_like(b_i) for b_i in b]

        for j in range(len(x_batch)):

          output = self.forward_prop(x_batch[j], W, b)
          dW_inter, db_inter = self.backword_prop(y = y_batch[j], x = x_batch[j], W = W, b = b)

          for i in range(len(W)):
            dW[i] += dW_inter[i]
            db[i] += db_inter[i]

       # average dw and db in one batch
        dW = [dw / len(x_batch) for dw in dW]
        db = [db_i / len(x_batch) for db_i in db]

        for i in range(len(W)):
          dv_W[i] = self.beta_RMS*dv_W[i] + (1-self.beta_RMS)*(dW[i])**2
          dv_b[i] = self.beta_RMS*dv_b[i] + (1-self.beta_RMS)*(db[i])**2




       #calculating W, b for 1 batch
        for i in range(len(W)):
          W[i] -= self.learning_rate/np.sqrt(dv_W[i] + self.epsilon) * dW[i]
          b[i] -= self.learning_rate/np.sqrt(dv_b[i] + self.epsilon) * db[i]



      # acccuracy and loss calculated after each epoch
      acuu, loss = self.accuracyLoss(W = W, b = b, x = self.x_train, y = self.y_train)
      v_acc, v_loss = self.accuracyLoss(W=  W, b = b, x = self.x_val, y = self.y_val)

      if self.isWandb == True:
        wandb.log({'accuracy': acuu})
        wandb.log({'loss': loss})
        wandb.log({'v_accuracy': v_acc})
        wandb.log({'v_loss': v_loss})
        wandb.log({'epoch': iteration})
      print(f" accuracy = {acuu}, Loss = {loss}, Validation_accuracy =  {v_acc}, Validation_loss =  {v_loss}")
      iteration += 1


    # True W, b after training

    return W, b





  def ADAM(self):
    W, b = self.weights, self.bias
    iteration = 0
    t = 1
    dv_W = [np.zeros_like(w) for w in self.weights]
    dv_b = [np.zeros_like(b_i) for b_i in self.bias]
    dm_W = [np.zeros_like(w) for w in self.weights]
    dm_b = [np.zeros_like(b_i) for b_i in self.bias]
    m_hat_W = [np.zeros_like(w) for w in self.weights]
    m_hat_b = [np.zeros_like(b_i) for b_i in self.bias]
    v_hat_W = [np.zeros_like(w) for w in self.weights]
    v_hat_b = [np.zeros_like(b_i) for b_i in self.bias]

    for epoch in range(self.epochs):
      print(f'epoch {epoch + 1}/{self.epochs}')



      indices = np.arange(len(self.x_train))
      np.random.shuffle(indices)

      x_train_shuffle = self.x_train[indices]
      y_train_shuffle = self.y_train[indices]

      for i in range(0, len(x_train_shuffle), self.batch_size):
        x_batch = x_train_shuffle[i:i + self.batch_size]
        y_batch = y_train_shuffle[i:i + self.batch_size]

        # Initilise dw and db
        dW = [np.zeros_like(w) for w in W]
        db = [np.zeros_like(b_i) for b_i in b]

        for j in range(len(x_batch)):

          output = self.forward_prop(x_batch[j], W, b)
          dW_inter, db_inter = self.backword_prop(y = y_batch[j], x = x_batch[j], W = W, b = b)

          for i in range(len(W)):
            dW[i] += dW_inter[i]
            db[i] += db_inter[i]

       # average dw and db in one batch
        dW = [dw / len(x_batch) for dw in dW]
        db = [db_i / len(x_batch) for db_i in db]

        for i in range(len(W)):
          dv_W[i] = self.beta2*dv_W[i] + (1-self.beta2)*(dW[i])**2
          dv_b[i] = self.beta2*dv_b[i] + (1-self.beta2)*(db[i])**2

          v_hat_W[i] = dv_W[i]/(1 - self.beta2**t)
          v_hat_b[i] = dv_b[i]/(1 - self.beta2**t)


        for i in range(len(W)):
          dm_W[i] = self.beta1*dm_W[i] + (1-self.beta1)*dW[i]
          dm_b[i] = self.beta1*dm_b[i] + (1-self.beta1)*db[i]

          m_hat_W[i] = dm_W[i]/(1 - self.beta1**t)
          m_hat_b[i] = dm_b[i]/(1 - self.beta1**t)




       #calculating W, b for 1 batch
        for i in range(len(W)):
          W[i] -= self.learning_rate/np.sqrt(v_hat_W[i] + self.epsilon) * m_hat_W[i]
          b[i] -= self.learning_rate/np.sqrt(v_hat_b[i] + self.epsilon) * m_hat_b[i]





        t += 1
      # acccuracy and loss calculated after each epoch
      acuu, loss = self.accuracyLoss(W = W, b = b, x = self.x_train, y = self.y_train)
      v_acc, v_loss = self.accuracyLoss(W =  W, b = b, x = self.x_val, y = self.y_val)

      if self.isWandb == True:
        wandb.log({'accuracy': acuu})
        wandb.log({'loss': loss})
        wandb.log({'v_accuracy': v_acc})
        wandb.log({'v_loss': v_loss})
        wandb.log({'epoch': iteration})
      print(f" accuracy = {acuu}, Loss = {loss}, Validation_accuracy =  {v_acc}, Validation_loss =  {v_loss}")
      iteration += 1


    # True W, b after training

    return W, b











  def optimseWeights(self):
    if self.optimiser == "sgd":
      self.SGD()
    if self.optimiser == "momentum":
      self.Momentam_GD()
    if self.optimiser == "nag":
      self.Nestrov_Based_GD()
    if self.optimiser == "rmsprop":
      self.RMS_prop()
    if self.optimiser == "adam":
      self.ADAM()
    #if self.optimiser == "nadam":
      #self.NADAM()













































# Define the object of class with number of inputs mlayers and number of inputs (change as needed)
#layer_sizes = [784, 3, 4, 2, 10]
#nn = neural_network( epochs = 5, layers = layer_sizes, learning_rate = 0.01, batch_size = 10 ,  optimiser= 'some', activationFunc= 'sigmoid')

# Test with one sample image
#sample_image = x_test[:10]
#actual_output = y_test[:10]
#output_probs = nn.backword_prop(y = actual_output[i] for i in range(len(sample_image)), x = sample_image[i] for i in range(len(sample_image)) )

# Display image and predicted class
#plt.imshow(sample_image.reshape(28, 28), cmap='gray')
#plt.title(f"Predicted output Class: {np.argmax(output_probs)}")
#plt.axis('off')
#plt.show()
#print("Actual output class : " , actual_output)
#print("Output layer : ", output_probs)






In [4]:
NN = neural_network(layers = [784,32,32,32,32,10], learning_rate = 1e-3 ,weightDecay = 0.0005, batch_size = 32, epochs = 10, optimiser = 'adam', activationFunc = "relu", beta = 0.1, beta_RMS = 0.9)
NN.optimseWeights()

epoch 1/10
 accuracy = 59.42777777777778, Loss = [6.78975574], Validation_accuracy =  58.95, Validation_loss =  [6.90440254]
epoch 2/10
 accuracy = 59.60925925925926, Loss = [4.71425065], Validation_accuracy =  58.8, Validation_loss =  [4.74684644]
epoch 3/10
 accuracy = 63.19259259259259, Loss = [2.61754802], Validation_accuracy =  62.06666666666667, Validation_loss =  [2.62847844]
epoch 4/10
 accuracy = 62.412962962962965, Loss = [2.02337856], Validation_accuracy =  61.71666666666667, Validation_loss =  [2.06048749]
epoch 5/10
 accuracy = 63.06111111111111, Loss = [1.55208413], Validation_accuracy =  62.25, Validation_loss =  [1.60014303]
epoch 6/10
 accuracy = 67.54814814814814, Loss = [1.10645223], Validation_accuracy =  66.81666666666666, Validation_loss =  [1.19544417]
epoch 7/10
 accuracy = 70.1962962962963, Loss = [0.97860867], Validation_accuracy =  69.0, Validation_loss =  [1.06633386]
epoch 8/10
 accuracy = 72.42037037037036, Loss = [0.88686855], Validation_accuracy =  72.18

In [14]:
# Import the W&B Python Library and log into W&B
import wandb



def main():
    wandb.init(project = "DA6401_ASSIGNMENT_1", entity="devyadav11")
    config = wandb.config
    run_name = f"{config.optimiser}_{config.activationFunc}_{config.layers}_{config.batch_size}_{config.epochs}_{config.learning_rate}"

    # Set the run name
    wandb.run.name = run_name
    wandb.run.save()

    # Define and train the model as before
    model = neural_network(layers = config.layers, learning_rate = config.learning_rate, weightDecay = config.weightDecay, batch_size = config.batch_size, epochs = config.epochs, optimiser = config.optimiser,
                           activationFunc = config.activationFunc, beta = 0.1, beta_RMS = 0.9, isWandb = True)
    model.optimseWeights()
    wandb.finish()

sweep_config = {
    'method': 'bayes',
    'name' : 'sweep cross entropy',
    'metric': {
      'name': 'v_accuracy',
      'goal': 'maximize'
    },
    'parameters': {
        'epochs': {
            'values': [5,10]
        },
        'layers': {
          'values': [[784,32,32,32,10],[784,32,32,32,32,10],[784,32,32,32,32,32,10],[784,64,64,64,10],[784,64,64,64,64,10],[784,64,64,64,64,64,10],[784,128,128,128,10],[784,128,128,128,128,10],[784,128,128,128,128,128,10]]
        },
        'weight_decay': {
            'values':[0, 0.0005, 0.5]
        },
        'batch_size': {
            'values': [16, 32, 64]
        },
        'activation': {
            'values': ['sigmoid','relu','tanh']
        },
        'optimiser': {
            'values': ['sgd', 'momentum', 'nestrov', 'rmsprop', 'adam', 'nadam']
        },

        'learning_rate': {
            'values':[1e-2,1e-3,1e-4]
        },
    }
}


sweep_id = wandb.sweep(sweep=sweep_config,project='Assignment 1')
wandb.agent(sweep_id, function = main , count = 400)

wandb.finish()


Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f317748e910>> (for pre_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: y1sjndm3
Sweep URL: https://wandb.ai/devyadav11/Assignment%201/sweeps/y1sjndm3


wandb: Agent Starting Run: il24wh1n with config:
wandb: 	activation: relu
wandb: 	batch_size: 64
wandb: 	epochs: 5
wandb: 	layers: [784, 32, 32, 32, 32, 32, 10]
wandb: 	learning_rate: 0.001
wandb: 	optimiser: nadam
wandb: 	weight_decay: 0.5


...

Exception in thread Thread-84 (_run_job):
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/wandb/agents/pyagent.py", line 306, in _run_job
    self._function()
  File "<ipython-input-14-d83d0d53fec0>", line 7, in main
  File "/usr/local/lib/python3.11/dist-packages/wandb/sdk/wandb_init.py", line 1482, in init
    wandb._sentry.reraise(e)
  File "/usr/local/lib/python3.11/dist-packages/wandb/analytics/sentry.py", line 156, in reraise
    raise exc.with_traceback(sys.exc_info()[2])
  File "/usr/local/lib/python3.11/dist-packages/wandb/sdk/wandb_init.py", line 1468, in init
    return wi.init(run_settings, run_config)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/wandb/sdk/wandb_init.py", line 779, in init
    with telemetry.context() as tel:
  File "/usr/local/lib/python3.11/dist-packages/wandb/sdk/lib/telemetry.py", line 42, in __exit__
    self._run._telemetry_callback(self._obj)
  File "/usr/local/lib/pyt

Exception in thread Thread-85 (_run_job):
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/wandb/agents/pyagent.py", line 306, in _run_job
    self._function()
  File "<ipython-input-14-d83d0d53fec0>", line 7, in main
  File "/usr/local/lib/python3.11/dist-packages/wandb/sdk/wandb_init.py", line 1482, in init
    wandb._sentry.reraise(e)
  File "/usr/local/lib/python3.11/dist-packages/wandb/analytics/sentry.py", line 156, in reraise
    raise exc.with_traceback(sys.exc_info()[2])
  File "/usr/local/lib/python3.11/dist-packages/wandb/sdk/wandb_init.py", line 1468, in init
    return wi.init(run_settings, run_config)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/wandb/sdk/wandb_init.py", line 779, in init
    with telemetry.context() as tel:
  File "/usr/local/lib/python3.11/dist-packages/wandb/sdk/lib/telemetry.py", line 42, in __exit__
    self._run._telemetry_callback(self._obj)
  File "/usr/local/lib/pyt

BrokenPipeError: [Errno 32] Broken pipe

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f317748e910>> (for post_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

In [ ]:
print(nn.Momentam_GD())

epoch 1/5
29.07037037037037 [1.57334886] 27.75 [1.58260031]
epoch 2/5
33.50925925925926 [1.5548366] 32.416666666666664 [1.56577405]
epoch 3/5
34.93888888888889 [1.5394559] 33.11666666666667 [1.55028018]
epoch 4/5
35.388888888888886 [1.52443971] 33.68333333333333 [1.53718896]
epoch 5/5
36.1 [1.51172667] 35.1 [1.5261773]
([array([[-0.27070228,  0.10489139,  0.25013566, ...,  1.94964318,
         0.40494763, -0.16494107],
       [-0.94577805, -0.30444262, -0.21545299, ..., -0.31944781,
         0.06980257, -1.36961517],
       [-0.51654135, -0.24696814, -2.20140528, ..., -0.54053154,
         0.68417043, -0.79623549]]), array([[-3.60543532,  3.65616138, -1.45845399],
       [ 1.30297555, -3.41037239, -0.50574993],
       [ 0.5155856 , -2.67281197, -1.79651272],
       [ 0.22827634, -1.88292553, -1.14066795]]), array([[-2.87217765,  3.72921043,  2.92370059,  0.25934627],
       [-6.89453463,  2.76512291,  1.09963528,  1.19061775]]), array([[-0.90600851, -3.66033028],
       [-1.8263397 , -

In [ ]:
print(nn.Nestrov_Based_GD())

epoch 1/5
26.455555555555556 [1.6118486] 26.6 [1.62068211]
epoch 2/5
37.25185185185185 [1.46217695] 35.75 [1.47732705]
epoch 3/5
38.148148148148145 [1.41142422] 36.583333333333336 [1.435723]
epoch 4/5
38.416666666666664 [1.39204789] 36.95 [1.42185156]
epoch 5/5
38.27037037037037 [1.36653839] 37.35 [1.38858843]
([array([[-0.27059244,  0.10482111,  0.24817831, ...,  1.86579524,
         0.36138395, -0.18683263],
       [-0.9455031 , -0.30426542, -0.21407914, ..., -0.40770434,
         0.01672431, -1.37382024],
       [-0.51655937, -0.24670112, -2.20125858, ..., -0.54874818,
         0.64394122, -0.79842974]]), array([[-3.86360172,  3.95004897, -1.55402195],
       [ 1.60682353, -3.90867573, -1.80290942],
       [ 0.68410715, -3.57039326, -2.81614615],
       [ 0.49911869, -1.9886957 , -1.08984625]]), array([[-4.15959822,  4.59947622,  4.44428734,  0.80534656],
       [-7.8797717 ,  3.23296708,  1.08242903,  1.2043629 ]]), array([[-0.98408013, -4.24129673],
       [-1.86805408, -5.6574668

In [ ]:
print(nn.ADAM())

epoch 1/5
39.242592592592594 [1.19724347] 39.65 [1.19790734]
epoch 2/5
39.42962962962963 [1.19674192] 38.7 [1.20601041]
epoch 3/5
39.157407407407405 [1.18708628] 39.38333333333333 [1.18924113]
epoch 4/5
39.266666666666666 [1.18594743] 39.56666666666667 [1.19325105]
epoch 5/5
39.25 [1.17933231] 40.05 [1.1901015]
([array([[-0.31898924,  0.07789128, -2.01388142, ..., -2.43850474,
        -3.02352726, -1.71479018],
       [-1.14886144, -0.46317882,  0.26803459, ...,  0.46327581,
         1.14474906, -1.04598443],
       [-0.51659134, -0.24715608, -2.39099208, ..., -1.44961304,
         0.02219246, -1.04690454]]), array([[-7.36163313, 13.9188818 ,  1.3100147 ],
       [ 2.77096203, -1.01295809, -4.32113941],
       [ 1.9029203 ,  4.3446815 , -4.4515171 ],
       [ 2.99692242, -4.9834625 ,  1.42413562]]), array([[-3.79108993,  2.92679348,  5.19809079,  0.37668222],
       [-4.1134751 ,  4.90448949,  2.58761941,  4.27711854]]), array([[ -5.31882145,  -8.16592304],
       [ -3.73731033, -11.85

In [ ]:
import numpy as np

h0 = a2 =  [[0.09292387],
 [0.29251695]]
h0 = np.array(h0)  # Convert h0 to a NumPy array
d1 = h0*(1 - h0)
print(d1)


In [ ]:
print((np.zeros())))